<a href="https://colab.research.google.com/github/sharmapukar217/docker-images/blob/main/Clamphook_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
rm -rf * # to clear data; run this

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import os
import io
import pandas as pd
from google.colab import files

# utility to load data
def get_df_from_upload(uploaded):
  filename = next(iter(uploaded))
  ext = os.path.splitext(filename)[-1]
  if ext == ".csv":
    return pd.read_csv(io.BytesIO(list(uploaded.values())[0]) ,sep=",").drop_duplicates()
  elif ext == ".xls":
    return pd.read_excel(io.BytesIO(list(uploaded.values())[0])).drop_duplicates()
  else:
    raise "Unsupported file format: [.csv, .xls]"

set_a_answer = files.upload()
set_b_answer = files.upload()
submitted_answers = files.upload()

Saving set-a-acme-solution.xls to set-a-acme-solution.xls


Saving set-b-acme.xls to set-b-acme.xls


Saving set a acme omr.csv to set a acme omr (1).csv


In [50]:
questions = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70', 'Q71', 'Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79', 'Q80', 'Q81', 'Q82', 'Q83', 'Q84', 'Q85', 'Q86', 'Q87', 'Q88', 'Q89', 'Q90', 'Q91', 'Q92', 'Q93', 'Q94', 'Q95', 'Q96', 'Q97', 'Q98', 'Q99', 'Q100']

df_set_a_result = pd.DataFrame([], columns=["Symbol number", *questions, "score"])

answer_set_a = get_df_from_upload(set_a_answer)
answer_set_b = get_df_from_upload(set_b_answer)

df = get_df_from_upload(submitted_answers)

df.columns.values[1:101] = questions
df.columns = [*df.columns[:-1], 'symbol_number'] # rename last col to symbol number
df.dropna(subset=["symbol_number"], inplace=True) # remove null symbol number data

# get symbol number
symbol_number = df.loc[:,df.columns.str.contains('^roll')].fillna(0).agg(lambda x:''.join(x.astype(int).astype(str)), axis=1)
symbol_number.name="Symbol Number"

df_with_sn = df.loc[:,~df.columns.str.contains('^roll')].join(symbol_number)
df_with_sn = df_with_sn.set_index("Symbol Number")
df_with_sn=df_with_sn.drop(columns=["symbol_number"])

df_with_sn.columns.values[1:101] = questions
df_with_sn.drop(['File name'], axis=1, inplace=True)

if "File name" in answer_set_a:
  answer_set_a.drop(['File name'], axis=1, inplace=True)


df_with_sn_group_a = df_with_sn.iloc[:,0:60]
df_with_sn_group_b = df_with_sn.iloc[:,60:]

df_with_sn_group_a_nan = df_with_sn_group_a.isna().sum(axis=1)
df_with_sn_a_group_b_nan = df_with_sn_group_b.isna().sum(axis=1)

df_correct = df_with_sn.eq(answer_set_a.squeeze("index").values)

df_score = df_correct.iloc[:,0:60].sum(axis=1)+2*df_correct.iloc[:,60:].sum(axis=1)
df_score =df_score-(140-(df_score+df_with_sn_group_a_nan+2*df_with_sn_a_group_b_nan))*0.1


df_score.to_csv("score.csv")

